This challenge involves breaking a truncated LCG that outputs some MSBs and some LSBs of the current state. Some references for attacks:

- https://www.math.cmu.edu/~af1p/Texfiles/RECONTRUNC.pdf 
- https://crypto.stackexchange.com/questions/37836/problem-with-lll-reduction-on-truncated-lcg-schemes

The intended solution is to use lattice-based techniques to recover enough information about the state of the LCG to predict future outputs.

### Truncated LCG

The states $x_i$ of the LCG are related by the formula 

$$x_{i+1} \equiv ax_i + c \pmod M \qquad i \geq 1$$

Let $l$ be the number of bits in $M$. The truncated LCG outputs $2^{\text{trunc}} (x_{i} \mod 2^\text{trunc}) + \lfloor \frac{x_i}{2^{l-\text{trunc}}} \rfloor$ (i.e. the $\text{trunc}$ MSBs concatenated to the $\text{trunc}$ LSBs). The initial state $x_1$ is a large random number.

### Breaking the Truncated LCG: Recovering the State

Suppose we are given $k+1$ outputs of the LCG.

Let $w_i, y_i, z_i$ be such that $2^{l-\text{trunc}}w_i + 2^{\text{trunc}}y_i + z_i = x_i$. The $w_i$ and $z_i$ are the outputs of the truncated LCG that we are given.

In the challenge, we are given $a$ and $M$. To eliminate the unknown constanct $c$, let

$$x_i' = x_{i+1} - x_i$$

Notice that

$$\begin{aligned} x_i' & = x_{i+1} - x_i \\ &= 2^{l-\text{trunc}}w_{i+1} + 2^{\text{trunc}}y_{i+1} + z_{i+1} - 2^{l-\text{trunc}}w_i - 2^{\text{trunc}}y_i - z_i \\ &= 2^{l-\text{trunc}}(w_{i+1} - w_i) + 2^{\text{trunc}}(y_{i+1} - y_i) + (z_{i+1} - z_i) \end{aligned}$$

So it makes sense to define $w_i', y_i'$ and $z_i'$ by $2^{l-\text{trunc}}w_i' + 2^{\text{trunc}}y_i' + z_i' = x_i'$.

The $x_i'$ are related by

$$\begin{aligned} x_i' &\equiv x_{i+1} - x_i \\ &\equiv ax_i + c - (ax_{i-1} + c) \\ &\equiv a(x_i - x_{i-1}) \\ &\equiv ax_{i-1}' \pmod M \end{aligned}$$

Hence, $a^{i-1} x_1' - x_i' \equiv 0 \pmod M$ for $1 \leq i \leq k$.

Now let $\mathbf{x}' = (x_1', x_2', \ldots, x_k')$

Consider the lattice generated by the rows of the matrix

$$L = \begin{bmatrix} M & 0 & 0 & \cdots & 0 \\ a & -1 & 0 & \cdots & 0 \\ a^2 & 0 & -1 & \cdots & 0 \\ \vdots & \ddots & & & \vdots \\ a^{k-1} & 0 & 0 & \cdots & -1 \end{bmatrix}$$

Notice that $L \cdot \mathbf{x}' = (Mx_1', ax_1' - x_2', \ldots, a^{k-1}x_1' - x_k') \equiv \mathbf{0} \pmod M$.

Let $B$ be the reduced basis for the lattice.

So

$$\begin{aligned}
    B \cdot \mathbf{x}' &\equiv \mathbf{0} \pmod M \\
    \implies B \cdot (2^{l-\text{trunc}} \mathbf{w}' + 2^{\text{trunc}} \mathbf{y}' + \mathbf{z}') &\equiv \mathbf{0} \pmod M \\
    \implies B \cdot 2^{l-\text{trunc}}\mathbf{w}' + B \cdot 2^{\text{trunc}}\mathbf{y}' + B \cdot \mathbf{z}' &= \mathbf{v}M \quad \text{for some vector $\mathbf{v}$} \\
    \implies B \cdot \mathbf{y}' &= (\mathbf{v}M - 2^{l-\text{trunc}}B \cdot \mathbf{w}' - B \cdot \mathbf{z}')/2^{\text{trunc}}
\end{aligned}$$

Now since $B \cdot \mathbf{y}'$ is short, we expect $\mathbf{v}M - 2^{l-\text{trunc}} \mathbf{w}' - B \cdot \mathbf{z}'$ to be short too. So we take $\mathbf{v} = (v_1, v_2, \ldots, v_k)$ to be such that $\mathbf{v}M - 2^{l-\text{trunc}} B \cdot \mathbf{w}' - B \cdot \mathbf{z}'$ is short. In particular, we set

$$v_i = \lfloor (2^{l-\text{trunc}} B \mathbf{w}' + B \cdot \mathbf{z}')_i / M \rceil$$

We can then recover $\mathbf{y}'$ by solving the system of linear equations.

Then, we can recover the $x_i'$ with $x_i' = 2^{l-\text{trunc}} w_i' + 2^{\text{trunc}}y_i' + z_i'$.

### Breaking the Truncated LCG: Predicting Future Outputs

We have now recovered the differences between consecutive states of the LCG! The next step is to predict future outputs. It's unlikely we'll be able to recover $c$ directly since we only have the differences and not the actual states themselves. Fortunately, there's an easy way to predict the next output even without knowing $c$. The main thing to notice is that we aren't really interested in $x_{k+1}$; we don't need to recover the _full_ state, all we care about is the next output, which can be determined from $w_k$ and $z_k$.

So we have $x_{k-1}', w_k$ and $z_k$, and we want to find $w_{k+1}$ and $z_{k+1}$.

Notice that

$$x_k' = x_{k+1} - x_k$$

and

$$x_k' \equiv ax_{k-1}' \pmod M$$ 

So

$$ax_{k-1}' \mod M = (2^{l-\text{trunc}}w_{k+1} + 2^{\text{trunc}}y_{k+1} + z_{k+1}) - (2^{l-\text{trunc}}w_k + 2^{\text{trunc}}y_k + z_k)$$

Reducing modulo $2^{\text{trunc}}$, we get

$$\begin{aligned} ax_{k-1}' \mod M &\equiv z_{k+1} - z_k \pmod {2^{\text{trunc}}} \\ \implies z_{k+1} &\equiv (ax_{k-1}' \mod M) + z_k \pmod {2^{\text{trunc}}} \end{aligned}$$

And to determine $w_{k+1}$, notice that, in the equation

$$ax_{k-1}' \mod M = (2^{l-\text{trunc}}w_{k+1} + 2^{\text{trunc}}y_{k+1} + z_{k+1}) - (2^{l-\text{trunc}}w_k + 2^{\text{trunc}}y_k + z_k)$$

the top $\text{trunc}$ MSBs of $ax_{k-1}'$ is given by $w_{k+1} - w_k$, so we can determine $w_{k+1}$ by computing

$$w_{k+1} \equiv \lfloor (ax_{k-1}' \mod M)/2^{l-\text{trunc}} \rfloor + w_k \pmod{2^{\text{trunc}}}$$

So with that, we should be able to predict the next output of the LCG and grab the flag!

### A Subtle Mistake

If we try to implement the theory, we might be met with some unexpected results (speaking from experience...). You may have noticed in the previous section that one of the steps doesn't always hold.

By construction, we have $x_k' = x_{k+1} - x_k$ and $x_k' \equiv ax_{k-1}' \pmod M$. And we concluded from this that $ax_{k-1}' \mod M = x_{k+1} - x_k$. This conclusion isn't neccessarily true. For example, see what happens when $x_{k+1} - x_k$ is negative.

Since we can't determine $x_{k+1}$ and $x_k$ fully, we can't know whether $x_{k+1} - x_k$ is positive or negative. This means we may potentially get the wrong output. Luckily, the game lets us guess 5 numbers, and it turns out that this is enough for us to win every time.

If $x_{k+1} - x_k$ is positive, we can follow the steps above without modification and compute

$$\begin{aligned}
    z_{k+1} &\equiv (ax_{k-1}' \mod M) + z_k \pmod{2^{\text{trunc}}} \\
    w_{k+1} &\equiv \lfloor (ax_{k-1}' \mod M)/2^{l-\text{trunc}} \rfloor + w_k \pmod{2^{\text{trunc}}}
\end{aligned}$$

On the other hand, if $x_{k+1} - x_k$ is negative, then since $|x_{k+1} - x_k| < M$ and

$$ax_{k-1}' \equiv x_{k+1} - x_k \pmod{2^{\text{trunc}}}$$

 it must be true that

$$x_{k+1} - x_k = (ax_{k-1}' \mod M) - M$$

So in this case,

$$\begin{aligned}
    z_{k+1} &\equiv (ax_{k-1}' \mod M) - M + z_k \pmod{2^{\text{trunc}}} \\
    w_{k+1} &\equiv \lfloor ((ax_{k-1}' \mod M) - M)/2^{l-\text{trunc}} \rfloor + w_k \pmod{2^{\text{trunc}}}
\end{aligned}$$

For some reason (probably rounding errors), the guess is sometimes small by 1. We can work around this by sending two additional numbers, each adding 1 to what we found previously.

Maybe there's a way to determine the next output with 100% correctness that I'm not aware of. Please send me a message if you know!

### Solve Script

```python
import os
os.environ['PWNLIB_NOTERM'] = 'True'
from pwn import process, remote

def connect():
    return remote('0.0.0.0', 1337)
    # return process('../challenge/server.py', level='error')

def recv(conn):
    o = conn.recvline().decode()
    print('[<]', o)
    return o

def send(conn, data):
    print('[>]', data)
    conn.sendline(data)

def do_round(conn, trunc, guess=[1337]):
    recv(conn)
    send(conn, ' '.join(str(x) for x in guess))
    r = recv(conn)
    num = int(r.split('was ')[1])
    w = num % 2^trunc
    z = (num >> trunc) % 2^trunc
    return w, z

def get_params(conn):
    M = ZZ(recv(conn).split('M = ')[1])
    a = ZZ(recv(conn).split('a = ')[1])
    return M, a

def get_next_lcg_outputs(xprime, wk, zk, a, M, trunc):
    def r(xp):
        zk_next = (xp + zk) % 2^trunc
        wk_next = ((xp >> (M.nbits() - trunc)) + wk) % 2^trunc
        return (zk_next << trunc) + wk_next

    outputs = []

    xp = (a*xprime)%M
    outputs.append(r(xp))
    outputs.append(r(xp - M))
    outputs.append(r(xp) + 1)
    outputs.append(r(xp - M) + 1)

    return outputs

def recover_Xprime(W, Z, a, M, trunc):
    k = len(Z)-1
    Wprime = [W[i+1] - W[i] for i in range(k)]
    Zprime = [Z[i+1] - Z[i] for i in range(k)]
        
    L = [[a^i] + [0]*(i-1) + [-1] + [0]*(k-1-i) for i in range(1, k)]
    L.insert(0, [M] + [0]*(k-1))
    L = matrix(L)

    B = L.LLL()

    P1 = 2^(M.nbits() - trunc) * B * vector(Wprime) + B * vector(Zprime)
    P2 = vector([(round(RR(p) / M) * M - p)/(2^trunc) for p in P1])

    Yprime = list(B.solve_right(P2))
    Xprime = [2^(M.nbits() - trunc)*Wprime[i] + (2^trunc * Yprime[i]) + Zprime[i] for i in range(k)]
    return list(map(ZZ, Xprime))

def play_game():
    conn = connect()
    recv(conn)
    recv(conn)
    M, a = get_params(conn)
    trunc = 20

    WZ = [do_round(conn, trunc), do_round(conn, trunc)]
    W = [wz[0] for wz in WZ]
    Z = [wz[1] for wz in WZ]

    Xprime = recover_Xprime(W, Z, a, M, trunc)

    for _ in range(23-2):
        w, z = do_round(conn, trunc, get_next_lcg_outputs(Xprime[-1], W[-1], Z[-1], a, M, trunc))
        W.append(w)
        Z.append(z)
        Xprime = recover_Xprime(W, Z, a, M, trunc)

    recv(conn)

play_game()
```